## Step 2, Clean Data, Export to csv file
* Read data and create dataframe from csv file saved from Step1
* Remove columns that are not needed
* Clean data by replace terms
* Export the result to csv file, main_customers.csv, under data folder

In [1]:
# import all the dependencies 
import requests
import csv
import json
import pandas as pd
import requests, json
import re
import configparser
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Define all variables 
configFile = "config.cfg"
config = configparser.ConfigParser()
config.read(configFile)

['config.cfg']

In [3]:
print("Reading all the combined data , may take a few minutes")
customers_df = pd.read_csv("data/combined_data.csv", low_memory=False)
print('Shape of exports read from CSV : {}'.format(customers_df.shape))
customers_df.columns

Reading all the combined data , may take a few minutes
Shape of exports read from CSV : (753494, 35)


Index(['Unnamed: 0', 'EventModDead', 'EventModError', 'EventModFrustrated',
       'EventStart', 'EventTargetSelectorTok', 'EventTargetText', 'EventType',
       'IndvId', 'PageActiveDuration', 'PageAgent', 'PageBrowser',
       'PageDevice', 'PageDuration', 'PageId', 'PageIp', 'PageLatLong',
       'PageNumErrors', 'PageNumInfos', 'PageNumWarnings',
       'PageOperatingSystem', 'PageRefererUrl', 'PageUrl', 'SessionId',
       'UserAppKey', 'UserDisplayName', 'UserEmail', 'UserId',
       'user_anonymousId_str', 'user_firstName_str', 'user_lastName_str',
       'user_serviceName_str', 'user_userHash_str', 'user_userId_str',
       'user_username_str'],
      dtype='object')

In [4]:
# Dropping rows with extra services we don't need
print("Size of data before dropping servcies {}".format(customers_df.shape))
service_picked= ['LOG_INTELLIGENCE','CloudAssembly','SDK']
customers_df = customers_df[customers_df.user_serviceName_str.isin(service_picked)]
print("Size of data after  dropping servcies {} ".format(customers_df.shape))

Size of data before dropping servcies (753494, 35)
Size of data after  dropping servcies (711157, 35) 


In [5]:
# Replace service names to make them more meaningful
customers_df.replace("LOG_INTELLIGENCE", "Data Logging",inplace=True)
customers_df.replace('CloudAssembly', "Data Package",inplace=True)
print("Size of data {}".format(customers_df.shape))

Size of data (711157, 35)


In [6]:
customers_df.columns

Index(['Unnamed: 0', 'EventModDead', 'EventModError', 'EventModFrustrated',
       'EventStart', 'EventTargetSelectorTok', 'EventTargetText', 'EventType',
       'IndvId', 'PageActiveDuration', 'PageAgent', 'PageBrowser',
       'PageDevice', 'PageDuration', 'PageId', 'PageIp', 'PageLatLong',
       'PageNumErrors', 'PageNumInfos', 'PageNumWarnings',
       'PageOperatingSystem', 'PageRefererUrl', 'PageUrl', 'SessionId',
       'UserAppKey', 'UserDisplayName', 'UserEmail', 'UserId',
       'user_anonymousId_str', 'user_firstName_str', 'user_lastName_str',
       'user_serviceName_str', 'user_userHash_str', 'user_userId_str',
       'user_username_str'],
      dtype='object')

In [7]:
# Masking Some Sensitive URL
print("Masking start please wait, it may take a few minutes")
customers_df.replace("/www.mgmt.cloud.vmware.com/", "/www.cloud.dataguru.com/", inplace=True, regex=True)
customers_df.replace("vmware","dataguru", inplace=True,regex=True)
customers_df.replace("csp","bin", inplace =True,regex=True)
customers_df.replace("deployment","dataType",inplace =True,regex=True)
customers_df.replace("automation-ui","dataType-UI",inplace =True,regex=True)
customers_df.replace("provisioning","data-provision",inplace =True,regex=True)
print("Masking done")
print("Size of data {}".format(customers_df.shape))

Masking start please wait, it may take a few minutes
Masking done
Size of data (711157, 35)


In [8]:
customers_df.columns

Index(['Unnamed: 0', 'EventModDead', 'EventModError', 'EventModFrustrated',
       'EventStart', 'EventTargetSelectorTok', 'EventTargetText', 'EventType',
       'IndvId', 'PageActiveDuration', 'PageAgent', 'PageBrowser',
       'PageDevice', 'PageDuration', 'PageId', 'PageIp', 'PageLatLong',
       'PageNumErrors', 'PageNumInfos', 'PageNumWarnings',
       'PageOperatingSystem', 'PageRefererUrl', 'PageUrl', 'SessionId',
       'UserAppKey', 'UserDisplayName', 'UserEmail', 'UserId',
       'user_anonymousId_str', 'user_firstName_str', 'user_lastName_str',
       'user_serviceName_str', 'user_userHash_str', 'user_userId_str',
       'user_username_str'],
      dtype='object')

In [9]:
customers_df.shape

(711157, 35)

In [10]:
# Add a column as Domain 
print('Adding domain column')
customers_df['Domain'] = customers_df['UserEmail'].str.split(pat="@", expand=True)[1]
print('Shape of exports read from CSV : {}'.format(customers_df.shape))

Adding domain column
Shape of exports read from CSV : (711157, 36)


In [11]:
# drop rows that doesn't have an email (invalid data)
customers_df = customers_df.dropna(subset=['UserEmail'])

# Read the info on emails fron config file 
dropemails= config.get('Emails', 'dropEmailList')
dropemailsList = dropemails.split(",")
external_customers_df = customers_df[~ customers_df.Domain.isin(dropemailsList)]

print('Shape of dataframe after dropping internal users : {}'.format(external_customers_df.shape))

Shape of dataframe after dropping internal users : (78291, 36)


In [12]:
# Dropping unwanted columns 
external_customers_df = external_customers_df[['Domain','UserAppKey','user_serviceName_str','UserId','IndvId','SessionId',
                                       'PageUrl','PageRefererUrl','PageBrowser','PageDevice','PageDuration','PageActiveDuration',
                                        'PageLatLong',
                                       'EventStart','EventType']]
print('Shape of dataframe after dropping extra columns: {}'.format(external_customers_df.shape))  

Shape of dataframe after dropping extra columns: (78291, 15)


In [13]:
# Renaming columns
external_customers_df.rename(columns={'UserAppKey':'UserEmail','user_serviceName_str':'serviceName'},inplace = True)
external_customers_df.columns

Index(['Domain', 'UserEmail', 'serviceName', 'UserId', 'IndvId', 'SessionId',
       'PageUrl', 'PageRefererUrl', 'PageBrowser', 'PageDevice',
       'PageDuration', 'PageActiveDuration', 'PageLatLong', 'EventStart',
       'EventType'],
      dtype='object')

In [14]:
# Printing list of unique services we have collected data for 
list(external_customers_df.serviceName.unique())

['Data Logging', 'SDK', 'Data Package']

In [15]:
# Masking Company Names 
domainList  = list(external_customers_df.Domain.unique())
counter = 1
for domain in domainList  :
    newDomain = "Company" + str(counter)+".com"
#     print(newDomain)
    external_customers_df.replace(domain,newDomain,inplace =True,regex=True)
    counter=counter +1  

print("Masking done")

Masking done


In [16]:
## Data Cleaning and filtering 
#dropping an Column not needed 
print('Number of columns before :{}'.format(len(customers_df.columns)))
customers_df.drop('Unnamed: 0', axis=1, inplace=True)
print('Number of columns after :{}'.format(len(customers_df.columns)))

Number of columns before :36
Number of columns after :35


In [17]:
customers_df.columns

Index(['EventModDead', 'EventModError', 'EventModFrustrated', 'EventStart',
       'EventTargetSelectorTok', 'EventTargetText', 'EventType', 'IndvId',
       'PageActiveDuration', 'PageAgent', 'PageBrowser', 'PageDevice',
       'PageDuration', 'PageId', 'PageIp', 'PageLatLong', 'PageNumErrors',
       'PageNumInfos', 'PageNumWarnings', 'PageOperatingSystem',
       'PageRefererUrl', 'PageUrl', 'SessionId', 'UserAppKey',
       'UserDisplayName', 'UserEmail', 'UserId', 'user_anonymousId_str',
       'user_firstName_str', 'user_lastName_str', 'user_serviceName_str',
       'user_userHash_str', 'user_userId_str', 'user_username_str', 'Domain'],
      dtype='object')

In [20]:
# Writing clean data to CSV
external_customers_df.to_csv("data/external_customers.csv", index=False)
print("external Customer data Successfully written to external_customers.csv ")

external Customer data Successfully written to external_customers.csv 
